In [1]:
import pandas as pd
import numpy as np
import vaex
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [2]:
traf = vaex.open('../../TRAFICO/traf_hist_unir.hdf5')

acci_filtered_vaex = vaex.open('../../acci_filtered_vaex.hdf5')

accis_id = acci_filtered_vaex.to_pandas_df()

ids = list(set(accis_id.idelem.value_counts().index).intersection(traf.idelem.unique()))

acci = pd.read_csv('../data/biblio_acci.csv')


In [4]:
ids

Index(['4206', '4436', '4774', '3901', '4452', '5828', '4170', '3798', '5484',
       '4166',
       ...
       '5217', '6608', '5388', '6266', '6420', '10443', '10406', '10584',
       '10462', '5426'],
      dtype='object', length=4371)

In [3]:
import tqdm

for id in tqdm.tqdm(ids[20:2000]):

    #filtrar por idelem

    
    traf1 = traf[traf.idelem == id]

    traf2 = traf1.to_pandas_df()

    traf_acci = pd.merge(traf2, acci, on='unir', how='left')

    traf_acci['accidente'] = traf_acci['accidente'].fillna(0)

    if traf_acci['accidente'].sum() > 0:
        
        traf_acci.drop(['unir', 'idelem'], axis=1, inplace=True)

        traf_acci['fecha'] = traf_acci['fecha_full'].str.split(' ').str[0]
            
        traf_acci['hora'] = traf_acci['fecha_full'].str.split(' ').str[1]

        traf_acci.drop('fecha_full', axis=1, inplace=True)

        traf_acci['prec'] = pd.to_numeric(traf_acci['prec'].str.replace(',', '.'), errors='coerce')

        traf_acci['velmedia'] = pd.to_numeric(traf_acci['velmedia'].str.replace(',', '.'), errors='coerce')

        traf_acci.fillna(0, inplace=True)

        traf_acci['mes'] = traf_acci['fecha'].str.split('-').str[1]

        traf_acci['fecha'] = pd.to_datetime(traf_acci['fecha'])

        traf_acci['dia_semana'] = traf_acci['fecha'].dt.dayofweek

        traf_acci.drop('fecha', axis=1, inplace=True)

        traf_acci['dia_semana'] = traf_acci['dia_semana'].astype('object')

                #get dummies

        traf_acci_dum = pd.get_dummies(traf_acci, columns=['mes', 'dia_semana', 'hora'])

                #Xtest, ytest

        X = traf_acci_dum.drop('accidente', axis=1)

        y = traf_acci_dum['accidente']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                #model

        model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
        model.fit(X_train, y_train)

        filename = 'modelo_' + id + '.sav'
        pickle.dump(model, open('../modelos_svm/' + filename, 'wb'))#generar tabla para modelo





  0%|          | 0/1980 [00:00<?, ?it/s]c:\Users\calba\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
  0%|          | 2/1980 [02:35<39:15:12, 71.44s/it] c:\Users\calba\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
  0%|          | 3/1980 [03:09<29:45:41, 54.19s/it]c:\Users\calba\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
  0%|          | 4/1980 [04:35<36:36:13, 66.69s/it]c:\Users\calba\anaconda3\lib\site-packages\pandas\cor

1